# Emojis Visualizations - Fake Covid-19 dataset

We've used the following packages:

In [1]:
import pandas as pd
import numpy as np
import json
import itertools  
from collections import Counter
import altair as alt
import demoji
import csv

#demoji.download_codes()

To read the JSON file that has all the tweets and to read the CSV to do the classification later, it is necessary to do:

In [2]:
csv_dataframe = pd.read_csv('dataset/FINAL_fakecovid_final_filtered_dataset_clean.csv',sep=";")
csv_dataframe['tweet_id'] = csv_dataframe['tweet_id'].astype(str)
csv_list = csv_dataframe.values.tolist()
lista_unica_csv=list(itertools.chain.from_iterable(csv_list))

data = []
with open('dataset/fakecovid_result_final_translated_full.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

We're interested in the "full_text" field and "id_str" (for the tweets classification):

In [3]:
index= 0
emos = []
for element in data:
    token_id = data[index]['id_str']
    indice_csv = lista_unica_csv.index(token_id)
    em = demoji.findall_list(data[index]['full_text'], False)
    if em:
        for e in em: 
            final_token = e + " " + lista_unica_csv[indice_csv+1].lower().replace(" ", "")
            emos.append(final_token)
    index= index+1
   
    

We have calculated the frequency distribution and worked on the dataframe to display the correct data:

In [4]:
#merged=list(itertools.chain.from_iterable(emos))
#print(merged)
#print(emos)
c = Counter(emos)


df = pd.DataFrame.from_dict(c, orient='index').reset_index()
df = df.rename(columns={'index':'Emojis', 0:'count'})
col_one_list = df['Emojis'].tolist()
col_two_list = df['count'].tolist()

typelist=[]
namelist=[]

index = 0

count_false = [0] * len(col_one_list)
count_part = [0] * len(col_one_list)

for el in col_one_list:
    tok = el.split()
    namelist.append(tok[0])
    if tok[0] in namelist:
        indx = namelist.index(tok[0])
        if tok[1] == "false":
            count_false[indx] = col_two_list[index]
        elif tok[1] == "partiallyfalse":
            count_part[indx] = col_two_list[index]
        else:
            print("errore count")
   
    index = index + 1

i=0
for el in col_two_list:
    col_two_list[i] = count_false[i] + count_part[i]
    i = i + 1

df['Emojis']=namelist
df['False']=count_false
df['Partially False']=count_part
df['count'] = col_two_list


df = df.sort_values(by=['count'],ascending=[False])

    Emojis  count  False  Partially False
57      ⚠️     26     26                0
14      ⬇️     25     25                0
8        👇     19     14                5
0        🚨     15     13                2
55       🔴     13     12                1
41       🙏     12      9                3
67       🤔     12     10                2
7        ❌     11     10                1
60       ✅     10      9                1
26       👉      9      9                0
18      🇺🇸      9      7                2
65       ⚠      8      8                0
48      ➡️      7      7                0
21       📢      7      7                0
86      ▶️      6      1                5
32      🇮🇹      6      3                3
56      🇨🇳      6      5                1
147      👏      6      0                6
13       😂      6      6                0
126      💪      5      2                3
9       ✔️      5      5                0
31      🇪🇸      5      5                0
169     👏🏽      5      5          

Here it is the final result, the chart displays only the emoji appeared in at least five different tweets:

In [5]:
bars = alt.Chart(df).transform_fold(
    ['False', 'Partially False'],
).mark_bar().encode(
    x=alt.X('key:N',title=None), # axis=alt.Axis(labels=False)
    y=alt.Y('value:Q',title='Tweet count'),
    color=alt.Color('key:N',scale=alt.Scale(range=['#5D3A9B','#E66100']),title="Category"),
    column=alt.Column("Emojis:N")
).transform_filter(
    alt.FieldRangePredicate(field='count', range=[5, 26])
).properties(
        title="The most frequent emojis in tweets classified by category (False, Partually False)",
).configure_title(
    fontSize=17,
    offset=25
).configure_axis(
    labelFontSize=13,
    titleFontSize=15,
    titlePadding=15
).configure_legend(
    titleFontSize=14,
    labelFontSize=12,
    titlePadding=10
    #orient="left"
).configure_header(
    titleFontSize=15,
    labelFontSize=12
)

bars

alt.Chart(...)